In [9]:
import pymskt as mskt
from pymskt.mesh import BoneMesh, CartilageMesh
import os
import numpy as np

In [2]:
location_seg = '../../data/right_knee_example.nrrd'

In [44]:
fem_cart = CartilageMesh(
    path_seg_image=location_seg,  # path to the segmentation iamge being used
    label_idx=1,                  # what is the label of this bone
)

fem_cart.create_mesh()
fem_cart.resample_surface(clusters=10000)

# initiate the bone mesh object
femur = BoneMesh(path_seg_image=location_seg,  # path to the segmentation iamge being used
                 label_idx=5,                  # what is the label of this bone
                #  list_cartilage_labels=[1])    # a list of labels for cartialge associated w/ this bone
                list_cartilage_meshes=[fem_cart]
)

# Create the bone mesh
femur.create_mesh(
    smooth_image_var=1.0       # This is the variance of the gaussian filter applied to binary image b4 meshing
)

# Resample the bone surface to have a specified number of nodes. 
femur.resample_surface(
    clusters=10000             # This is the number of nodes/vertices on the surface. It might vary slightly
)

# Calcualte cartialge thickness for the cartialge meshes associated with the bone
femur.calc_cartilage_thickness(
    image_smooth_var_cart=0.5   # variance for the gaussian filter applied to the binary cart meshes b4 smoothing. 
)

INTERSECTION IS:  2


In [49]:
n_points = 100

# Get 10 points on bone surface with cartilage thickness
thickness = femur.get_scalar('thickness (mm)')
cart_points = np.where(thickness > 0.0)
# get 10 rand points. 
cart_points = np.random.choice(cart_points[0], n_points)

# get points xyz positions 
bone_origin = femur.point_coords[cart_points]

# Project vector normal to surface for these points. 
from pymskt.mesh.utils import get_surface_normals
normals = get_surface_normals(femur.mesh)
normals = normals.GetOutput().GetPointData().GetNormals()
normals = np.asarray([normals.GetTuple(idx) for idx in cart_points])
# Get intersection points on cartilage for these points.
from pymskt.mesh.utils import get_obb_surface, get_intersect, get_arrow
import pyvista as pv
obb_cartilage = get_obb_surface(femur.list_cartilage_meshes[0].mesh)

dict_vectors = {}

length = 10
length_opposite = 1

for idx, point in enumerate(bone_origin):
    vector = normals[idx]
    start_point_ray = point - length_opposite * vector
    end_point_ray = point + length * vector
    points_intersect, cell_ids_intersect = get_intersect(obb_cartilage, start_point_ray, end_point_ray)

    arrow = get_arrow(
        vector,
        start_point_ray,
        scale=length + length_opposite,
        tip_length=0.25,
        tip_radius=0.1,
        tip_resolution=20, 
        shaft_radius=0.01,
        shaft_resolution=20
    )

    spheres = [pv.Sphere(center=point, radius=0.5) for point in points_intersect]

    dict_vectors[idx] = {
        'start': start_point_ray, 
        'end': end_point_ray, 
        'points_intersect': points_intersect,
        'arrow': arrow,
        'spheres': spheres
    }


# draw line for the vector between these two points. 

In [34]:
from itkwidgets import Viewer

# Note that we have to get the mesh from the femur object `femur.mesh` to input it into itkwidgets. 
# In this notebook, you should be able to swap between scalars (thickness & labels)
# thickness is the cartilage thickness in mm, labels is the cartilage label of the associated thickness value. 
Viewer(geometries=[femur.mesh, femur.list_cartilage_meshes[0].mesh, dict_vectors[0]['arrow']])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [50]:
# View with just vectors
list_geometries = [femur.mesh, femur.list_cartilage_meshes[0].mesh]
list_arrows = [dict_vectors[idx]['arrow'] for idx in dict_vectors]
list_geometries.extend(list_arrows)
Viewer(geometries=list_geometries)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [52]:
n_vectors = 10

# View with just vectors
list_geometries = [femur.mesh, femur.list_cartilage_meshes[0].mesh]
list_arrows = [dict_vectors[idx]['arrow'] for idx in range(n_vectors)]
list_spheres = [dict_vectors[idx]['spheres'][i] for idx in range(n_vectors) for i in range(2)]
list_geometries.extend(list_arrows)
list_geometries.extend(list_spheres)

Viewer(geometries=list_geometries)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [54]:
Viewer(geometries=[femur.mesh, femur.list_cartilage_meshes[0].mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…